In [1]:
using GeoStats
import GeoIO: load
import CairoMakie: barplot, density, set_theme!, theme_dark
import DataFrames: filter, leftjoin, groupby, combine, transform!
using GLM
include("./utils/load_utils.jl")
set_theme!(theme_dark())

In [2]:
incidents =
    load("data/incidents.geojson") |>
    Map(:law_cat_cd => map_law_cat => "law_cat") |>
    Map([:ofns_desc, :law_cat] => map_crime_cat => "crime_cat") |>
    Filter(filter_crime_types) |>
    Select("crime_cat" => "crime") |>
    OneHot("crime")
incidents = georef(DataFrame(values(incidents)), domain(incidents))


crime_BURGLARY,crime_DRUGS,crime_LARCENY,crime_VIOLENCE,geometry
Categorical,Categorical,Categorical,Categorical,Point
[NoUnits],[NoUnits],[NoUnits],[NoUnits],🖈 GeodeticLatLon{WGS84Latest}
false,false,true,false,"(lat: 40.7528°, lon: -73.9715°)"
false,false,false,true,"(lat: 0.0°, lon: 0.0°)"
false,false,true,false,"(lat: 40.7547°, lon: -73.9868°)"
false,false,false,true,"(lat: 40.752°, lon: -73.9774°)"
false,false,true,false,"(lat: 40.7772°, lon: -73.9881°)"
false,false,true,false,"(lat: 40.7329°, lon: -73.9813°)"
false,false,true,false,"(lat: 40.7541°, lon: -73.9939°)"
false,false,true,false,"(lat: 40.7461°, lon: -73.9789°)"


In [3]:
arrests =
    load("data/arrests.geojson") |>
    DropMissing() |>
    Map(:law_cat_cd => map_law_cat => "law_cat") |>
    Map([:ofns_desc, :law_cat] => map_crime_cat => "crime_cat") |>
    Filter(filter_crime_types) |>
    Select("crime_cat" => "crime") |>
    OneHot("crime")
arrests = georef(DataFrame(values(arrests)), domain(arrests))

crime_BURGLARY,crime_DRUGS,crime_LARCENY,crime_VIOLENCE,geometry
Categorical,Categorical,Categorical,Categorical,Point
[NoUnits],[NoUnits],[NoUnits],[NoUnits],🖈 GeodeticLatLon{WGS84Latest}
false,false,false,true,"(lat: 40.8148°, lon: -73.9563°)"
false,false,false,true,"(lat: 40.8148°, lon: -73.9575°)"
false,false,false,true,"(lat: 40.7845°, lon: -73.9755°)"
false,false,false,true,"(lat: 40.7568°, lon: -73.9711°)"
false,false,false,true,"(lat: 40.7252°, lon: -73.9997°)"
false,false,false,true,"(lat: 40.8148°, lon: -73.9563°)"
false,true,false,false,"(lat: 40.7961°, lon: -73.9435°)"
true,false,false,false,"(lat: 40.7308°, lon: -74.0057°)"


In [4]:
places =
    build_place_table("data/places/") |>
    DropMissing(["primary_type"]) |>
    Map(:primary_type => map_top_place_category => "top_cat") |>
    Select("top_cat" => "category") |>
    OneHot("category")
places = georef(DataFrame(values(places)), domain(places))

category_,category_AUTOMOTIVE,category_BUSINESS,category_CULTURE,category_EDUCATION,category_FINANCE,category_FOODBEV,category_GOVERNMENT,category_HEALTH,category_LODGING,category_RECREATION,category_SERVICES,category_SHOPPING,category_TRANSPORTATION,category_WORSHIP,geometry
Categorical,Categorical,Categorical,Categorical,Categorical,Categorical,Categorical,Categorical,Categorical,Categorical,Categorical,Categorical,Categorical,Categorical,Categorical,Point
[NoUnits],[NoUnits],[NoUnits],[NoUnits],[NoUnits],[NoUnits],[NoUnits],[NoUnits],[NoUnits],[NoUnits],[NoUnits],[NoUnits],[NoUnits],[NoUnits],[NoUnits],🖈 GeodeticLatLon{WGS84Latest}
false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,"(lat: 40.7935°, lon: -73.9717°)"
false,true,false,false,false,false,false,false,false,false,false,false,false,false,false,"(lat: 40.8361°, lon: -73.9396°)"
false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,"(lat: 40.8164°, lon: -73.9454°)"
false,false,false,false,false,false,false,false,false,false,true,false,false,false,false,"(lat: 40.8001°, lon: -73.9422°)"
false,false,false,false,false,false,false,false,true,false,false,false,false,false,false,"(lat: 40.7161°, lon: -74.0126°)"
false,false,false,false,false,false,true,false,false,false,false,false,false,false,false,"(lat: 40.7189°, lon: -73.991°)"
false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,"(lat: 40.734°, lon: -74.0018°)"
false,false,false,false,false,false,false,false,false,false,false,true,false,false,false,"(lat: 40.7259°, lon: -73.9931°)"


In [5]:
streets =
    load("data/streets.geojson") |>
    Filter(filter_streets_manhattan) |>
    Select("physicalid" => "street_id")
streets = georef(DataFrame(values(streets)), domain(streets))


street_id,geometry
Categorical,Multi
[NoUnits],🖈 GeodeticLatLon{WGS84Latest}
3,Multi(1×Rope)
5,Multi(1×Rope)
6,Multi(1×Rope)
8,Multi(1×Rope)
14,Multi(1×Rope)
23,Multi(1×Rope)
24,Multi(1×Rope)
25,Multi(1×Rope)


In [ ]:
### spatial joins to match point data with streets
incidents = match_points_streets!(incidents, streets)
places = match_points_streets!(places, streets)
arrests = match_points_streets!(arrests, streets)
median_street_dist_center = calc_median_street_dist_center(streets.geometry)

In [ ]:
### group and aggregate by street
incidents = combine(groupby(incidents, :street_id),
    :crime_VIOLENCE => sum,
    :crime_LARCENY => sum,
    :crime_DRUGS => sum,
    :crime_BURGLARY => sum)

places = combine(groupby(places, :street_id),
    :category_AUTOMOTIVE => sum,
    :category_BUSINESS => sum,
    :category_CULTURE => sum,
    :category_EDUCATION => sum,
    :category_FINANCE => sum,
    :category_FOODBEV => sum,
    :category_GOVERNMENT => sum,
    :category_HEALTH => sum,
    :category_LODGING => sum,
    :category_RECREATION => sum,
    :category_SERVICES => sum,
    :category_SHOPPING => sum,
    :category_TRANSPORTATION => sum,
    :category_WORSHIP => sum)

arrests = combine(groupby(arrests, :street_id),
    :crime_VIOLENCE => sum,
    :crime_LARCENY => sum,
    :crime_DRUGS => sum,
    :crime_BURGLARY => sum)


In [ ]:
### join aggregate tables back to streets
streets = tablejoin(streets, incidents, kind=:left, on=:street_id)
streets = tablejoin(streets, places, kind=:left, on=:street_id)
streets = tablejoin(streets, arrests, kind=:left, on=:street_id)
### convert missing values to zero counts
streets = georef(coalesce.(values(streets), 0), streets.geometry)